## Expand All

In [11]:
# iframe_fixed_scraper.py
"""
FIXED: Iframe Context and Button Clicking Issues
This version properly handles iframe switching for button clicks and URL resolution
"""

import os
import time
import uuid
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import json
import random
import re

# Configuration
SCREENSHOTS_DIR = "screenshots"
DATA_FILE = "privacy_notice_data.xlsx"
MAX_DEPTH = 3
WAIT_TIME = 5

os.makedirs(SCREENSHOTS_DIR, exist_ok=True)

class IframeFixedPrivacyScraper:
    def __init__(self, headless=False):
        self.driver = self.initialize_driver_with_stealth(headless)
        self.data = []
        self.current_iframe = None  # Track if we're in an iframe
        
    def initialize_driver_with_stealth(self, headless=False):
        """Enhanced driver initialization"""
        chrome_options = Options()
        
        if headless:
            chrome_options.add_argument("--headless")
            
        # Enhanced anti-detection measures
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        
        # Rotate user agents
        user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0"
        ]
        user_agent = random.choice(user_agents)
        chrome_options.add_argument(f"user-agent={user_agent}")
        
        # Initialize driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        # Execute script to mask automation
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        print(f"Driver initialized with user agent: {user_agent[:50]}...")
        return driver
    
    def wait_for_consent_banner(self, timeout=10):
        """Enhanced banner detection"""
        print(f"Waiting up to {timeout}s for consent banner to appear...")
        
        # Strategy 1: Common CMP indicators
        cmp_indicators = [
            ("id", "onetrust-banner-sdk"),
            ("id", "onetrust-consent-sdk"),
            ("class", "ot-sdk-container"),
            ("id", "CybotCookiebotDialog"),
            ("class", "CybotCookiebotDialog"),
            ("id", "truste-consent-track"),
            ("class", "truste-banner"),
            ("class", "qc-cmp2-container"),
            ("class", "qc-cmp-ui-container"),
            ("id", "didomi-popup"),
            ("class", "didomi-popup-container"),
            ("id", "usercentrics-root"),
            ("data-testid", "uc-container"),
            ("class", "cookie-consent"),
            ("class", "cookie-banner"),
            ("class", "consent-banner"),
            ("class", "privacy-banner"),
            ("class", "gdpr-banner"),
            ("role", "dialog"),
        ]
        
        for attr_type, attr_value in cmp_indicators:
            try:
                if attr_type == "id":
                    element = WebDriverWait(self.driver, 2).until(
                        EC.presence_of_element_located((By.ID, attr_value))
                    )
                elif attr_type == "class":
                    element = WebDriverWait(self.driver, 2).until(
                        EC.presence_of_element_located((By.CLASS_NAME, attr_value))
                    )
                elif attr_type == "data-testid":
                    element = WebDriverWait(self.driver, 2).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, f"[data-testid='{attr_value}']"))
                    )
                elif attr_type == "role":
                    element = WebDriverWait(self.driver, 2).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, f"[role='{attr_value}']"))
                    )
                
                if element and element.is_displayed():
                    print(f"Found consent banner via {attr_type}='{attr_value}'")
                    return True
            except TimeoutException:
                continue
        
        # Strategy 2: Check for iframes
        try:
            iframes = self.driver.find_elements(By.TAG_NAME, "iframe")
            for iframe in iframes:
                iframe_attrs = {
                    "id": iframe.get_attribute("id") or "",
                    "class": iframe.get_attribute("class") or "",
                    "src": iframe.get_attribute("src") or "",
                    "name": iframe.get_attribute("name") or ""
                }
                
                consent_keywords = ['consent', 'cookie', 'privacy', 'gdpr', 'ccpa', 'cmp', 'notice']
                if any(keyword in str(iframe_attrs).lower() for keyword in consent_keywords):
                    print(f"Found potential consent iframe: {iframe_attrs}")
                    return True
        except Exception as e:
            print(f"Error checking iframes: {e}")
        
        return False
    
    def find_consent_elements(self):
        """Enhanced element detection with iframe tracking"""
        consent_elements = {
            "banners": [],
            "iframes": [],
            "buttons": [],
            "links": [],
            "toggles": []
        }
        
        # Reset iframe tracking
        self.current_iframe = None
        
        # Enhanced selectors for consent banners
        banner_selectors = [
            "[id*='cookie'][id*='banner']",
            "[id*='cookie'][id*='consent']",
            "[id*='privacy'][id*='banner']",
            "[id*='gdpr']",
            "[id*='consent']",
            "#onetrust-banner-sdk",
            "#CybotCookiebotDialog",
            "#didomi-popup",
            "#usercentrics-root",
            "[class*='cookie-banner']",
            "[class*='consent-banner']",
            "[class*='privacy-banner']",
            "[class*='gdpr-banner']",
            ".qc-cmp2-container",
            ".ot-sdk-container",
            "[role='dialog'][aria-label*='cookie']",
            "[role='dialog'][aria-label*='consent']",
            "[role='dialog'][aria-label*='privacy']",
            "[data-testid*='cookie']",
            "[data-testid*='consent']",
            "[data-component*='cookie']",
            "[data-component*='consent']"
        ]
        
        # Find banner elements
        for selector in banner_selectors:
            try:
                elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    if element.is_displayed() and element.size['height'] > 50:
                        text = element.text.lower()
                        if any(keyword in text for keyword in ['cookie', 'privacy', 'consent', 'data', 'accept']):
                            consent_elements["banners"].append(element)
                            print(f"Found banner via selector: {selector}")
            except Exception as e:
                continue
        
        # Find iframes and store the active one
        try:
            iframes = self.driver.find_elements(By.TAG_NAME, "iframe")
            for iframe in iframes:
                iframe_info = self._get_iframe_info(iframe)
                if self._is_consent_iframe(iframe_info):
                    consent_elements["iframes"].append(iframe)
                    # Store the iframe for later use
                    self.current_iframe = iframe
        except Exception:
            pass
        
        return consent_elements
    
    def _get_iframe_info(self, iframe):
        """Get iframe attributes"""
        return {
            "id": iframe.get_attribute("id") or "",
            "class": iframe.get_attribute("class") or "",
            "src": iframe.get_attribute("src") or "",
            "name": iframe.get_attribute("name") or ""
        }
    
    def _is_consent_iframe(self, iframe_info):
        """Check if iframe likely contains consent content"""
        consent_keywords = ['consent', 'cookie', 'privacy', 'gdpr', 'ccpa', 'cmp', 'notice']
        iframe_str = str(iframe_info).lower()
        return any(keyword in iframe_str for keyword in consent_keywords)
    
    def extract_notice_details(self, element):
        """Extract text, buttons and links from a notice element"""
        details = {
            "text": element.text.strip(),
            "buttons": [],
            "links": [],
            "is_in_iframe": False
        }
        
        # Extract buttons
        try:
            buttons = element.find_elements(By.TAG_NAME, "button")
            for button in buttons:
                if button.is_displayed():
                    button_text = button.text.strip()
                    if button_text:
                        details["buttons"].append({
                            "text": button_text,
                            "element": button
                        })
        except Exception as e:
            print(f"Error extracting buttons: {e}")
        
        # Also look for span/div buttons
        try:
            spans_divs = element.find_elements(By.CSS_SELECTOR, "span[role='button'], div[role='button']")
            for item in spans_divs:
                if item.is_displayed():
                    item_text = item.text.strip()
                    if item_text:
                        details["buttons"].append({
                            "text": item_text,
                            "element": item
                        })
        except Exception as e:
            print(f"Error extracting span/div buttons: {e}")
        
        # Extract links
        try:
            links = element.find_elements(By.TAG_NAME, "a")
            for link in links:
                if link.is_displayed():
                    link_text = link.text.strip()
                    link_href = link.get_attribute("href")
                    if link_text and link_href:
                        # Resolve iframe URLs to actual policy URLs
                        resolved_href = self._resolve_policy_url(link_href, link_text)
                        details["links"].append({
                            "text": link_text,
                            "href": resolved_href,
                            "original_href": link_href,
                            "element": link
                        })
        except Exception as e:
            print(f"Error extracting links: {e}")
        
        return details
    
    def _resolve_policy_url(self, href, link_text):
        """GENERALIZED URL resolution - works for any website"""
        # If it's already a normal URL (not iframe), return as is
        if href.startswith("http") and "iframe" not in href and "#" not in href:
            return href
        
        # Get base domain from current page
        current_url = self.driver.current_url
        parsed = urlparse(current_url)
        base_domain = f"{parsed.scheme}://{parsed.netloc}"
        
        # If it's an iframe URL, try to extract the actual link from iframe content
        if "iframe" in href or href.startswith("#"):
            # First, try to find the actual link by looking in the iframe
            try:
                if self.current_iframe:
                    # We're already in iframe context, look for actual links
                    actual_links = self.driver.find_elements(By.CSS_SELECTOR, "a[href]:not([href*='iframe'])")
                    
                    for actual_link in actual_links:
                        if actual_link.is_displayed():
                            actual_text = actual_link.text.strip().lower()
                            link_text_lower = link_text.lower()
                            
                            # Match based on text similarity
                            if (link_text_lower in actual_text or 
                                actual_text in link_text_lower or
                                self._text_similarity(actual_text, link_text_lower) > 0.6):
                                
                                actual_href = actual_link.get_attribute("href")
                                if actual_href and actual_href.startswith("http"):
                                    print(f"🔗 Found actual link for '{link_text}': {actual_href}")
                                    return actual_href
            except Exception as e:
                print(f"⚠️ Could not find actual link in iframe: {e}")
        
        # Fallback: Generate likely URL based on common patterns
        link_lower = link_text.lower()
        
        # Common policy URL patterns for any website
        if "privacy policy faq" in link_lower or ("faq" in link_lower and "privacy" in link_lower):
            # Try common FAQ URL patterns
            common_faq_patterns = [
                "/help/privacy-faq",
                "/support/privacy-faq", 
                "/privacy/faq",
                "/help/security/privacy-policy-faq",
                "/legal/privacy-faq",
                "/privacy-faq"
            ]
            # Return the first pattern (can be enhanced to test which exists)
            return f"{base_domain}{common_faq_patterns[0]}"
            
        elif "privacy policy" in link_lower or "privacy" in link_lower:
            return f"{base_domain}/privacy"
            
        elif "cookie policy" in link_lower or "cookie" in link_lower:
            return f"{base_domain}/cookie-policy"
            
        elif "terms" in link_lower:
            return f"{base_domain}/terms"
            
        elif "legal" in link_lower:
            return f"{base_domain}/legal"
        else:
            # Generic fallback
            return f"{base_domain}/privacy"
    
    def _text_similarity(self, text1, text2):
        """Simple text similarity calculation"""
        if not text1 or not text2:
            return 0.0
        
        # Convert to sets of words
        words1 = set(text1.lower().split())
        words2 = set(text2.lower().split())
        
        # Calculate Jaccard similarity
        intersection = words1.intersection(words2)
        union = words1.union(words2)
        
        if not union:
            return 0.0
        
        return len(intersection) / len(union)
    
    def switch_to_iframe_and_extract(self, iframe):
        """Switch to iframe and extract content with proper context tracking"""
        try:
            print(f"🔄 Switching to iframe for content extraction...")
            self.driver.switch_to.frame(iframe)
            self.current_iframe = iframe  # Track that we're in iframe
            
            # Check for notice elements in iframe
            try:
                body = self.driver.find_element(By.TAG_NAME, "body")
                notice_details = self.extract_notice_details(body)
                notice_details["is_in_iframe"] = True
                
                print(f"📦 Extracted iframe content: {len(notice_details['buttons'])} buttons, {len(notice_details['links'])} links")
                
            except:
                notice_details = {"text": "", "buttons": [], "links": [], "is_in_iframe": True}
            
            # DON'T switch back to default content yet - stay in iframe for interactions
            return notice_details
        except Exception as e:
            print(f"❌ Error processing iframe: {e}")
            self.driver.switch_to.default_content()
            self.current_iframe = None
            return {"text": "", "buttons": [], "links": [], "is_in_iframe": False}
    
    def _click_element_safely_with_iframe_context(self, element):
        """Enhanced element clicking with iframe context awareness"""
        try:
            print(f"🎯 Attempting to click element (iframe context: {self.current_iframe is not None})")
            
            # If we have an iframe and we're not in it, switch to it
            if self.current_iframe and not self._is_in_iframe():
                print(f"🔄 Switching to iframe for button click...")
                self.driver.switch_to.frame(self.current_iframe)
                time.sleep(1)
                
                # Re-find the button in iframe context
                button_text = element.text if hasattr(element, 'text') else "Unknown"
                buttons = self.driver.find_elements(By.CSS_SELECTOR, "button, [role='button']")
                
                target_button = None
                for btn in buttons:
                    if btn.is_displayed() and btn.text.strip() == button_text:
                        target_button = btn
                        break
                
                if target_button:
                    element = target_button
                    print(f"✅ Found button in iframe: {button_text}")
                else:
                    print(f"❌ Button not found in iframe: {button_text}")
                    return False
            
            # Try multiple click strategies
            strategies = [
                ("Direct click", lambda: element.click()),
                ("JavaScript click", lambda: self.driver.execute_script("arguments[0].click();", element)),
                ("Scroll and click", self._scroll_and_click),
                ("Actions click", self._actions_click)
            ]
            
            for strategy_name, strategy_func in strategies:
                try:
                    print(f"   🚀 Trying: {strategy_name}")
                    if strategy_name in ["Scroll and click", "Actions click"]:
                        strategy_func(element)
                    else:
                        strategy_func()
                    print(f"   ✅ Success with: {strategy_name}")
                    return True
                except Exception as e:
                    print(f"   ❌ Failed {strategy_name}: {e}")
                    continue
            
            return False
            
        except Exception as e:
            print(f"❌ All click strategies failed: {e}")
            return False
    
    def _is_in_iframe(self):
        """Check if we're currently in an iframe"""
        try:
            # Try to access the main window - if we're in iframe, this will be different
            main_window = self.driver.execute_script("return window.top === window;")
            return not main_window
        except:
            return False
    
    def _scroll_and_click(self, element):
        """Scroll to element and click"""
        self.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
        time.sleep(0.5)
        element.click()
    
    def _actions_click(self, element):
        """Use Actions API to click"""
        from selenium.webdriver.common.action_chains import ActionChains
        ActionChains(self.driver).move_to_element(element).click().perform()
    
    def display_options(self, notice_details):
        """Display interactive options to the user"""
        print(f"\n--- Notice Text ---\n{notice_details['text']}\n")
        
        print("--- Available Buttons ---")
        for i, button in enumerate(notice_details['buttons']):
            print(f"{i+1}. {button['text']}")
        
        print("\n--- Available Links ---")
        for i, link in enumerate(notice_details['links']):
            print(f"{i+1}. {link['text']} => {link['href']}")
        
        print("\n--- Actions ---")
        print("To click a button, enter: B<number> (e.g., B1 for the first button)")
        print("To click a link, enter: L<number> (e.g., L1 for the first link)")
        print("To take a manual screenshot, enter: S")
        print("To find and expand dropdown sections, enter: E")
        print("To go back, enter: BACK")
        print("To finish exploration, enter: DONE")
        
        return input("Enter your choice: ")
    
    def take_screenshot(self, state_id=None, custom_name=None):
        """Take screenshot"""
        domain = urlparse(self.current_website).netloc.replace('.', '_') if hasattr(self, 'current_website') else "unknown"
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        if custom_name:
            filename = f"{domain}_{custom_name}_{timestamp}.png"
        elif state_id:
            filename = f"{domain}_state_{state_id}_{timestamp}.png"
        else:
            filename = f"{domain}_{timestamp}.png"
            
        filepath = os.path.join(SCREENSHOTS_DIR, filename)
        self.driver.save_screenshot(filepath)
        return filepath
    
    def record_state(self, website, parent_id, state_id, notice_details, screenshot_path, choice=None):
        """Record state"""
        # Format links for Excel
        links_data = {}
        for i, link in enumerate(notice_details['links'][:5], 1):
            links_data[f"Link {i}"] = link['text']
            links_data[f"Link {i} Detail"] = link['href']
        
        # Ensure all link columns exist
        for i in range(len(notice_details['links']) + 1, 6):
            links_data[f"Link {i}"] = None
            links_data[f"Link {i} Detail"] = None
        
        row = {
            "Website": website,
            "ParentID": parent_id,
            "StateID": state_id,
            "Snapshot": screenshot_path,
            "Text of Notice": notice_details['text'],
            "Choice Provided": notice_details['buttons'][0]['text'] if notice_details['buttons'] else "No choices detected",
            "ChoiceMade": choice,
            "In Iframe": notice_details.get('is_in_iframe', False),
            **links_data
        }
        
        self.data.append(row)
        return self.data
    
    def explore_interactively(self, website, max_depth=3):
        """Main interactive exploration method with proper iframe handling"""
        self.current_website = website
        print(f"🌐 Visiting {website}...")
        
        try:
            # Clear cookies and navigate
            self.driver.delete_all_cookies()
            self.driver.get(website)
            time.sleep(3)
            
            # Check for bot detection
            print("🤖 Checking for bot detection...")
            page_title = self.driver.title.lower()
            page_text = self.driver.find_element(By.TAG_NAME, "body").text.lower()
            
            bot_indicators = ["security check", "captcha", "bot check", "human verification", "please verify"]
            if any(indicator in page_title or indicator in page_text for indicator in bot_indicators):
                print("⚠️  BOT DETECTION DETECTED!")
                input("Please solve the captcha/verification in the browser window, then press Enter to continue...")
                print("✅ Continuing with scraping...")
                time.sleep(2)
            
            # Generate root state
            root_state_id = str(uuid.uuid4())[:8]
            screenshot_path = self.take_screenshot(root_state_id, "initial")
            print(f"📸 Initial screenshot: {screenshot_path}")
            
            # Wait for banner
            banner_found = self.wait_for_consent_banner()
            
            # Find notice elements
            consent_elements = self.find_consent_elements()
            
            if consent_elements["banners"]:
                print(f"🎯 Found {len(consent_elements['banners'])} banner elements")
                notice_details = self.extract_notice_details(consent_elements["banners"][0])
            elif consent_elements["iframes"]:
                print(f"🎯 Found {len(consent_elements['iframes'])} iframe elements")
                notice_details = self.switch_to_iframe_and_extract(consent_elements["iframes"][0])
            else:
                print("❌ No consent banner found")
                notice_details = {"text": "No cookie notice detected", "buttons": [], "links": []}
            
            # Record initial state
            self.record_state(website, None, root_state_id, notice_details, screenshot_path)
            
            # Start interactive exploration
            current_state_id = root_state_id
            parent_id = None
            depth = 0
            state_stack = []
            
            while depth < max_depth:
                choice = self.display_options(notice_details)
                
                if choice.upper() == "DONE":
                    print("✅ Exploration complete!")
                    break
                
                elif choice.upper() == "S":
                    manual_screenshot = self.take_screenshot(custom_name=f"manual_{datetime.now().strftime('%H%M%S')}")
                    print(f"📸 Manual screenshot: {manual_screenshot}")
                    continue
                
                elif choice.upper().startswith("B") and notice_details['buttons']:
                    # Handle button clicks with iframe awareness
                    try:
                        button_idx = int(choice[1:]) - 1
                        if 0 <= button_idx < len(notice_details['buttons']):
                            button = notice_details['buttons'][button_idx]
                            print(f"🖱️ Clicking button: {button['text']}")
                            
                            # Save current state for back navigation
                            state_stack.append((self.driver.current_url, current_state_id, parent_id, self.current_iframe))
                            
                            # Update state trackers
                            parent_id = current_state_id
                            current_state_id = str(uuid.uuid4())[:8]
                            
                            # Use iframe-aware clicking
                            if self._click_element_safely_with_iframe_context(button['element']):
                                print("✅ Button clicked successfully!")
                                time.sleep(WAIT_TIME)
                                
                                # Take screenshot
                                screenshot_path = self.take_screenshot(current_state_id, f"after_{button['text'].replace(' ', '_')}")
                                print(f"📸 Screenshot: {screenshot_path}")
                                
                                # Find new notice state (stay in iframe if that's where we are)
                                try:
                                    if self.current_iframe and self._is_in_iframe():
                                        # We're in iframe, extract content from here
                                        body = self.driver.find_element(By.TAG_NAME, "body")
                                        notice_details = self.extract_notice_details(body)
                                        notice_details["is_in_iframe"] = True
                                    else:
                                        # We're in main content
                                        new_consent_elements = self.find_consent_elements()
                                        
                                        if new_consent_elements["banners"]:
                                            notice_details = self.extract_notice_details(new_consent_elements["banners"][0])
                                        elif new_consent_elements["iframes"]:
                                            notice_details = self.switch_to_iframe_and_extract(new_consent_elements["iframes"][0])
                                        else:
                                            notice_details = {"text": "No notice detected after clicking", "buttons": [], "links": []}
                                except Exception as e:
                                    print(f"⚠️ Error extracting new state: {e}")
                                    notice_details = {"text": "Error extracting new state", "buttons": [], "links": []}
                                
                                # Record new state
                                self.record_state(website, parent_id, current_state_id, notice_details, 
                                                screenshot_path, button['text'])
                                depth += 1
                                
                            else:
                                print("❌ Failed to click button")
                                # Restore state trackers since click failed
                                current_state_id = parent_id
                                parent_id = state_stack[-1][2] if state_stack else None
                                state_stack.pop() if state_stack else None
                        else:
                            print(f"❌ Invalid button index: {button_idx}")
                    except ValueError:
                        print(f"❌ Invalid button choice: {choice}")
                
                elif choice.upper().startswith("L") and notice_details['links']:
                    # Handle link clicks
                    try:
                        link_idx = int(choice[1:]) - 1
                        if 0 <= link_idx < len(notice_details['links']):
                            link = notice_details['links'][link_idx]
                            print(f"🔗 Clicking link: {link['text']} ({link['href']})")
                            
                            # Save current state for back navigation
                            state_stack.append((self.driver.current_url, current_state_id, parent_id, self.current_iframe))
                            
                            # Update state trackers
                            parent_id = current_state_id
                            current_state_id = str(uuid.uuid4())[:8]
                            
                            try:
                                # Switch back to main content for navigation
                                if self.current_iframe:
                                    self.driver.switch_to.default_content()
                                    self.current_iframe = None
                                
                                # Navigate to resolved link
                                print(f"🌐 Navigating to: {link['href']}")
                                self.driver.get(link['href'])
                                time.sleep(WAIT_TIME)
                                
                                # Take screenshot
                                screenshot_path = self.take_screenshot(current_state_id, "policy_page")
                                print(f"📸 Screenshot: {screenshot_path}")
                                
                                # Extract policy content
                                policy_content = self.extract_policy_page_content()
                                
                                # Record new state
                                self.record_state(website, parent_id, current_state_id, policy_content,
                                                screenshot_path, f"Link: {link['text']}")
                                
                                # Update notice_details for next iteration
                                notice_details = policy_content
                                depth += 1
                                
                            except Exception as e:
                                print(f"❌ Error clicking link: {e}")
                        else:
                            print(f"❌ Invalid link index: {link_idx}")
                    except ValueError:
                        print(f"❌ Invalid link choice: {choice}")
                
                elif choice.upper() == "E":
                    # Enhanced expandable section detection
                    print("🔧 Enhanced expandable section detection...")
                    
                    # Find dialog container (prioritize iframe context if we're in one)
                    dialog = None
                    if self.current_iframe and self._is_in_iframe():
                        print("📦 Using iframe context for expansion detection")
                        try:
                            dialog = self.driver.find_element(By.TAG_NAME, "body")
                        except:
                            pass
                    
                    if not dialog:
                        print("📦 Searching for dialog container in main content")
                        dialog_selectors = [
                            "#onetrust-consent-sdk", ".cookie-notice", "#cookie-notice",
                            "div[role='dialog']", ".modal-dialog", ".consent-modal"
                        ]
                        
                        for selector in dialog_selectors:
                            try:
                                elements = self.driver.find_elements(By.CSS_SELECTOR, selector)
                                for el in elements:
                                    if el.is_displayed():
                                        dialog = el
                                        print(f"📦 Found dialog container: {selector}")
                                        break
                                if dialog:
                                    break
                            except:
                                continue
                    
                    if not dialog:
                        print("📦 Using body as container")
                        if self.current_iframe and self._is_in_iframe():
                            dialog = self.driver.find_element(By.TAG_NAME, "body")
                        else:
                            # Switch to iframe if we have one
                            if self.current_iframe:
                                self.driver.switch_to.frame(self.current_iframe)
                                dialog = self.driver.find_element(By.TAG_NAME, "body")
                                print("📦 Switched to iframe and using body")
                            else:
                                dialog = self.driver.find_element(By.TAG_NAME, "body")
                    
                    # Find expandable sections with enhanced detection
                    expandables = self.find_expandable_sections_enhanced(dialog)
                    
                    if expandables:
                        print(f"🎯 Found {len(expandables)} expandable sections:")
                        for i, exp in enumerate(expandables):
                            display_text = exp['text'][:50] + "..." if len(exp['text']) > 50 else exp['text']
                            print(f"   {i+1}. {display_text} ({exp['type']})")
                        
                        print("\n🔧 Options:")
                        print("   A - Expand ALL sections")
                        print("   1-9 - Expand specific section number")
                        print("   SKIP - Skip expansion and continue")
                        
                        exp_choice = input("Enter your choice: ").strip()
                        
                        if exp_choice.upper() == "A":
                            print("🚀 Expanding ALL sections...")
                            expanded_count = 0
                            for i, exp in enumerate(expandables):
                                print(f"\n--- Expanding {i+1}/{len(expandables)}: {exp['text'][:30]}... ---")
                                success = self.try_expand_element_enhanced(exp)
                                if success:
                                    expanded_count += 1
                                    screenshot = self.take_screenshot(custom_name=f"expanded_{i+1}_{exp['type']}")
                                    print(f"📸 Screenshot saved: {screenshot}")
                                else:
                                    print(f"⚠️ Could not expand section {i+1}")
                                time.sleep(1)  # Brief pause between expansions
                            
                            print(f"\n✅ Expansion complete! Successfully expanded {expanded_count}/{len(expandables)} sections")
                            
                            # Take a final screenshot showing all expanded sections
                            final_screenshot = self.take_screenshot(custom_name="all_expanded_final")
                            print(f"📸 Final screenshot with all expansions: {final_screenshot}")
                            
                        elif exp_choice.upper() == "SKIP":
                            print("⏭️ Skipping expansion...")
                            
                        else:
                            # Handle specific section selection
                            try:
                                idx = int(exp_choice) - 1
                                if 0 <= idx < len(expandables):
                                    exp = expandables[idx]
                                    print(f"\n🚀 Expanding selected section: {exp['text'][:50]}...")
                                    success = self.try_expand_element_enhanced(exp)
                                    if success:
                                        screenshot = self.take_screenshot(custom_name=f"expanded_section_{idx+1}")
                                        print(f"📸 Screenshot saved: {screenshot}")
                                        
                                        # After expanding one section, ask if they want to expand more
                                        while True:
                                            more_choice = input("\nExpand another section? (Y/N): ").strip().upper()
                                            if more_choice == "Y":
                                                # Show remaining sections
                                                print("\n🎯 Available sections:")
                                                for i, exp in enumerate(expandables):
                                                    display_text = exp['text'][:50] + "..." if len(exp['text']) > 50 else exp['text']
                                                    print(f"   {i+1}. {display_text} ({exp['type']})")
                                                
                                                next_choice = input("Enter section number (or A for all remaining): ").strip()
                                                if next_choice.upper() == "A":
                                                    print("🚀 Expanding all remaining sections...")
                                                    for i, exp in enumerate(expandables):
                                                        print(f"\n--- Expanding {i+1}/{len(expandables)}: {exp['text'][:30]}... ---")
                                                        success = self.try_expand_element_enhanced(exp)
                                                        if success:
                                                            screenshot = self.take_screenshot(custom_name=f"expanded_{i+1}")
                                                            print(f"📸 Screenshot: {screenshot}")
                                                        time.sleep(1)
                                                    break
                                                else:
                                                    try:
                                                        next_idx = int(next_choice) - 1
                                                        if 0 <= next_idx < len(expandables):
                                                            next_exp = expandables[next_idx]
                                                            print(f"\n🚀 Expanding: {next_exp['text'][:50]}...")
                                                            success = self.try_expand_element_enhanced(next_exp)
                                                            if success:
                                                                screenshot = self.take_screenshot(custom_name=f"expanded_{next_idx+1}")
                                                                print(f"📸 Screenshot: {screenshot}")
                                                        else:
                                                            print("❌ Invalid section number")
                                                    except ValueError:
                                                        print("❌ Invalid input - please enter a number")
                                            elif more_choice == "N":
                                                break
                                            else:
                                                print("❌ Please enter Y or N")
                                    else:
                                        print(f"⚠️ Could not expand section: {exp['text'][:50]}")
                                else:
                                    print(f"❌ Invalid selection. Please enter 1-{len(expandables)}, A, or SKIP")
                            except ValueError:
                                print("❌ Invalid input. Please enter a number, A for all, or SKIP")
                    else:
                        print("❌ No expandable sections found")
                    
                    continue
                
                elif choice.upper() == "BACK":
                    if state_stack:
                        prev_url, prev_state_id, prev_parent_id, prev_iframe = state_stack.pop()
                        print(f"⬅️ Going back to state {prev_state_id}...")
                        
                        try:
                            # Switch back to main content first
                            if self.current_iframe:
                                self.driver.switch_to.default_content()
                            
                            self.driver.get(prev_url)
                            time.sleep(WAIT_TIME)
                            
                            # Update state trackers
                            current_state_id = prev_state_id
                            parent_id = prev_parent_id
                            self.current_iframe = prev_iframe
                            depth -= 1 if depth > 0 else 0
                            
                            # Rediscover notice elements
                            consent_elements = self.find_consent_elements()
                            
                            if consent_elements["banners"]:
                                notice_details = self.extract_notice_details(consent_elements["banners"][0])
                            elif consent_elements["iframes"]:
                                notice_details = self.switch_to_iframe_and_extract(consent_elements["iframes"][0])
                            else:
                                notice_details = {"text": "No notice detected", "buttons": [], "links": []}
                            
                            screenshot_path = self.take_screenshot(current_state_id, "back")
                            print(f"📸 Screenshot: {screenshot_path}")
                            
                        except Exception as e:
                            print(f"❌ Error going back: {e}")
                    else:
                        print("❌ Cannot go back further (at root state)")
                        
        except Exception as e:
            print(f"❌ Error during exploration: {e}")
            return self.data
        finally:
            # Ensure we're back to main content
            try:
                if self.current_iframe:
                    self.driver.switch_to.default_content()
            except:
                pass
            
        return self.data
    
    def find_expandable_sections_enhanced(self, container):
        """GENERALIZED expandable section detection for any website"""
        expandables = []
        
        print("🔍 Looking for expandable sections with enhanced detection...")
        
        # Strategy 1: Look for common cookie category patterns (works for any site)
        common_categories = [
            # Essential categories
            "strictly necessary", "necessary", "essential", "required",
            "functional", "functionality", "performance", 
            "analytics", "analytical", "statistics", "statistical",
            
            # Marketing categories  
            "marketing", "advertising", "targeting", "targeted",
            "social media", "social", "personalization", "personalized",
            
            # Data categories
            "third party", "data collection", "tracking", "cookies",
            "do not sell", "do not share", "opt out", "privacy choices"
        ]
        
        for category in common_categories:
            try:
                # Look for elements containing category text (case insensitive)
                xpath_patterns = [
                    f".//*[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{category.lower()}')]",
                    f".//div[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{category.lower()}')]",
                    f".//span[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{category.lower()}')]"
                ]
                
                for xpath in xpath_patterns:
                    try:
                        elements = container.find_elements(By.XPATH, xpath)
                        for element in elements:
                            if element.is_displayed():
                                element_text = element.text.strip()
                                # Only add if text is reasonable length (not entire page)
                                if 5 < len(element_text) < 200 and category.lower() in element_text.lower():
                                    print(f"   Found category element: {element_text[:50]}...")
                                    
                                    # Look for clickable parent
                                    clickable_element = self._find_clickable_parent(element)
                                    
                                    expandables.append({
                                        "element": clickable_element or element,
                                        "text": element_text,
                                        "type": "category",
                                        "strategy": f"text_match:{category}"
                                    })
                                    break
                        
                        # If we found something for this category, move to next
                        if expandables and category.lower() in expandables[-1]["text"].lower():
                            break
                            
                    except Exception as e:
                        continue
                        
            except Exception as e:
                print(f"   Error searching for {category}: {e}")
        
        # Strategy 2: Look for accordion/collapsible patterns
        accordion_selectors = [
            ".accordion-header", "[aria-expanded]", "[data-toggle='collapse']",
            "details", "summary", ".expandable", ".collapsible",
            ".toggle", "[role='button']", ".category-header"
        ]
        
        for selector in accordion_selectors:
            try:
                elements = container.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    if element.is_displayed():
                        text = element.text.strip()
                        # Only add if contains relevant keywords
                        if text and any(keyword in text.lower() for keyword in [
                            "cookie", "necessary", "functional", "analytics", 
                            "advertising", "marketing", "social", "performance",
                            "targeting", "privacy", "data"
                        ]):
                            expandables.append({
                                "element": element,
                                "text": text[:100],
                                "type": "accordion", 
                                "strategy": selector
                            })
            except Exception as e:
                continue
        
        # Strategy 3: Look for toggle/switch elements
        toggle_selectors = [
            "input[type='checkbox']", "input[type='switch']", 
            "[role='switch']", "[role='checkbox']",
            ".toggle", ".switch", "[class*='toggle']", "[class*='switch']"
        ]
        
        for selector in toggle_selectors:
            try:
                elements = container.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    if element.is_displayed():
                        label_text = self._get_toggle_label(element)
                        if label_text and "unknown" not in label_text.lower():
                            expandables.append({
                                "element": element,
                                "text": f"Toggle: {label_text}",
                                "type": "toggle",
                                "strategy": selector
                            })
            except Exception as e:
                continue
        
        # Strategy 4: Look for OneTrust specific (if present)
        onetrust_selectors = [
            ".ot-accordion-layout.ot-cat-item", ".ot-cat-header", 
            ".ot-acc-hdr", "#onetrust-pc-sdk .category-item"
        ]
        
        for selector in onetrust_selectors:
            try:
                elements = container.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    if element.is_displayed():
                        text = element.text.strip()
                        if text:
                            expandables.append({
                                "element": element,
                                "text": text[:100],
                                "type": "onetrust",
                                "strategy": selector
                            })
            except Exception as e:
                continue
        
        # Remove duplicates based on text similarity
        unique_expandables = []
        for exp in expandables:
            is_duplicate = False
            for unique_exp in unique_expandables:
                if self._text_similarity(exp["text"], unique_exp["text"]) > 0.8:
                    is_duplicate = True
                    break
            if not is_duplicate:
                unique_expandables.append(exp)
        
        print(f"🎯 Total unique expandable sections found: {len(unique_expandables)}")
        return unique_expandables
    
    def _find_clickable_parent(self, element):
        """Find a clickable parent element"""
        try:
            current = element
            for _ in range(3):  # Go up to 3 levels
                parent = current.find_element(By.XPATH, "..")
                if parent.tag_name == "body":
                    break
                
                # Check if parent is clickable
                if (parent.get_attribute("onclick") or 
                    parent.get_attribute("role") == "button" or
                    "click" in parent.get_attribute("class").lower() if parent.get_attribute("class") else False):
                    return parent
                
                current = parent
        except:
            pass
        return None
    
    def _get_toggle_label(self, toggle_element):
        """Get label text for a toggle element"""
        try:
            # Try to find associated label
            toggle_id = toggle_element.get_attribute("id")
            if toggle_id:
                try:
                    label = self.driver.find_element(By.CSS_SELECTOR, f"label[for='{toggle_id}']")
                    return label.text.strip()
                except:
                    pass
            
            # Try parent element
            try:
                parent = toggle_element.find_element(By.XPATH, "..")
                parent_text = parent.text.strip()
                # Remove the toggle element's text to get just the label
                toggle_text = toggle_element.text.strip()
                if toggle_text:
                    parent_text = parent_text.replace(toggle_text, "").strip()
                return parent_text if parent_text else "Unknown toggle"
            except:
                pass
                
            return "Unknown toggle"
        except:
            return "Unknown toggle"
    
    def try_expand_element_enhanced(self, expandable_info):
        """GENERALIZED element expansion strategies for any website"""
        element = expandable_info["element"]
        element_type = expandable_info["type"]
        strategy = expandable_info["strategy"]
        
        print(f"🚀 Attempting to expand: {expandable_info['text'][:50]}...")
        print(f"   Type: {element_type}, Strategy: {strategy}")
        
        # Enhanced universal expansion strategies
        universal_strategies = [
            ("Direct click", lambda: self._click_element_safely_with_iframe_context(element)),
            ("JavaScript click", lambda: self.driver.execute_script("arguments[0].click();", element)),
            ("Double click", lambda: self._double_click_element(element)),
            ("Parent click", lambda: self._click_parent_element(element)),
            ("Sibling click", lambda: self._click_sibling_elements(element)),
            ("Actions click", lambda: self._actions_click(element)),
            ("Force show content", lambda: self._force_show_hidden_content(element))
        ]
        
        for strategy_name, strategy_func in universal_strategies:
            try:
                print(f"   Trying strategy: {strategy_name}")
                result = strategy_func()
                if result:
                    print(f"   ✅ Success with: {strategy_name}")
                    time.sleep(2)  # Wait longer for expansion animation
                    return True
                else:
                    print(f"   ❌ Failed: {strategy_name}")
            except Exception as e:
                print(f"   ❌ Error with {strategy_name}: {e}")
        
        # Element-type specific strategies
        if element_type == "toggle":
            return self._expand_toggle_element(element)
        elif element_type == "accordion":
            return self._expand_accordion_element(element)
        elif element_type == "onetrust":
            return self._expand_onetrust_element(element)
        elif element_type == "category":
            return self._expand_category_element(element)
        
        print(f"   ⚠️ All strategies failed for: {expandable_info['text'][:30]}")
        return False
    
    def _double_click_element(self, element):
        """Try double-clicking the element"""
        try:
            from selenium.webdriver.common.action_chains import ActionChains
            ActionChains(self.driver).double_click(element).perform()
            return True
        except:
            return False
    
    def _force_show_hidden_content(self, element):
        """Force show hidden content using JavaScript"""
        try:
            # Look for hidden content near this element
            script = """
            var element = arguments[0];
            
            // Find potential content containers
            var containers = [
                element.nextElementSibling,
                element.parentElement.querySelector('[style*="display: none"]'),
                element.parentElement.querySelector('[style*="height: 0"]'),
                element.parentElement.querySelector('[aria-hidden="true"]'),
                element.parentElement.querySelector('.collapsed'),
                element.parentElement.querySelector('.hidden')
            ];
            
            var shown = false;
            containers.forEach(function(container) {
                if (container) {
                    container.style.display = 'block';
                    container.style.height = 'auto';
                    container.style.maxHeight = 'none';
                    container.style.overflow = 'visible';
                    container.setAttribute('aria-hidden', 'false');
                    container.classList.remove('collapsed', 'hidden');
                    shown = true;
                }
            });
            
            return shown;
            """
            return self.driver.execute_script(script, element)
        except:
            return False
    
    def _expand_category_element(self, element):
        """Category-specific expansion for cookie categories"""
        try:
            print(f"   Trying category-specific expansion strategies")
            
            # Strategy 1: Look for expand/arrow buttons near the element
            try:
                parent = element.find_element(By.XPATH, "..")
                expand_buttons = parent.find_elements(By.CSS_SELECTOR, 
                    "[class*='expand'], [class*='arrow'], [class*='chevron'], [class*='toggle'], button, [role='button']")
                
                for btn in expand_buttons:
                    if btn.is_displayed() and btn != element:
                        print(f"   Found potential expand button: {btn.get_attribute('class')}")
                        if self._click_element_safely_with_iframe_context(btn):
                            return True
            except:
                pass
            
            # Strategy 2: Try clicking the element itself
            if self._click_element_safely_with_iframe_context(element):
                return True
            
            # Strategy 3: Look for associated toggles/checkboxes
            try:
                parent = element.find_element(By.XPATH, "..")
                toggles = parent.find_elements(By.CSS_SELECTOR, "input[type='checkbox'], input[type='radio'], [role='switch']")
                
                for toggle in toggles:
                    if toggle.is_displayed():
                        if self._click_element_safely_with_iframe_context(toggle):
                            return True
            except:
                pass
            
            # Strategy 4: Force expansion with JavaScript
            try:
                script = """
                var element = arguments[0];
                
                // Find and click any nearby clickable elements
                var clickables = element.parentElement.querySelectorAll('*[onclick], button, [role="button"], [tabindex]');
                for (var i = 0; i < clickables.length; i++) {
                    if (clickables[i] !== element) {
                        clickables[i].click();
                        return true;
                    }
                }
                
                // Try triggering common events
                ['click', 'mousedown', 'mouseup', 'focus'].forEach(function(eventType) {
                    var event = new Event(eventType, { bubbles: true });
                    element.dispatchEvent(event);
                });
                
                return false;
                """
                return self.driver.execute_script(script, element)
            except:
                pass
                
            return False
            
        except Exception as e:
            print(f"   ❌ Error with category strategies: {e}")
            return False
    
    def _expand_toggle_element(self, element):
        """Toggle-specific expansion"""
        try:
            print(f"   Trying toggle-specific click")
            return self._click_element_safely_with_iframe_context(element)
        except Exception as e:
            print(f"   ❌ Error clicking toggle: {e}")
            return False
    
    def _expand_accordion_element(self, element):
        """Accordion-specific expansion"""
        try:
            print(f"   Trying accordion-specific strategies")
            
            # Try aria-expanded manipulation
            try:
                current_expanded = element.get_attribute("aria-expanded")
                if current_expanded == "false":
                    self.driver.execute_script("arguments[0].setAttribute('aria-expanded', 'true');", element)
                    
                    # Also try to show associated content
                    content_id = element.get_attribute("aria-controls")
                    if content_id:
                        self.driver.execute_script(f"""
                            var content = document.getElementById('{content_id}');
                            if(content) {{
                                content.style.display = 'block';
                                content.style.maxHeight = 'none';
                            }}
                        """)
                    return True
            except:
                pass
            
            # Fallback to direct click
            return self._click_element_safely_with_iframe_context(element)
            
        except Exception as e:
            print(f"   ❌ Error with accordion strategies: {e}")
            return False
    
    def _expand_onetrust_element(self, element):
        """OneTrust-specific expansion strategies"""
        try:
            print(f"   Trying OneTrust-specific strategies")
            
            # Try clicking arrow element first
            try:
                arrow = element.find_element(By.CSS_SELECTOR, ".ot-arw-cntr, .ot-arrow, [class*='arrow']")
                if self._click_element_safely_with_iframe_context(arrow):
                    return True
            except:
                pass
            
            # Try direct element click
            if self._click_element_safely_with_iframe_context(element):
                return True
            
            # Try OneTrust DOM manipulation
            try:
                script = """
                var element = arguments[0];
                element.setAttribute('aria-expanded', 'true');
                
                var contentId = element.getAttribute('aria-controls');
                if(contentId) {
                    var content = document.getElementById(contentId);
                    if(content) {
                        content.style.display = 'block';
                        content.style.maxHeight = 'none';
                    }
                }
                
                var arrow = element.querySelector('.ot-arw-cntr');
                if(arrow) {
                    arrow.classList.add('ot-arw-up');
                    arrow.classList.remove('ot-arw-down');
                }
                """
                self.driver.execute_script(script, element)
                return True
            except:
                pass
            
            return False
            
        except Exception as e:
            print(f"   ❌ Error with OneTrust strategies: {e}")
            return False
    
    def _click_parent_element(self, element):
        """Try clicking parent elements"""
        try:
            parent = element.find_element(By.XPATH, "..")
            return self._click_element_safely_with_iframe_context(parent)
        except:
            return False
    
    def _click_sibling_elements(self, element):
        """Try clicking sibling elements that might be toggles"""
        try:
            parent = element.find_element(By.XPATH, "..")
            siblings = parent.find_elements(By.XPATH, "./*")
            
            for sibling in siblings:
                if sibling != element and sibling.is_displayed():
                    # Look for toggle-like siblings
                    if (sibling.tag_name.lower() in ["button", "input"] or
                        sibling.get_attribute("role") in ["button", "switch", "checkbox"] or
                        "toggle" in sibling.get_attribute("class").lower() if sibling.get_attribute("class") else False):
                        
                        if self._click_element_safely_with_iframe_context(sibling):
                            return True
            return False
        except:
            return False
    
    def extract_policy_page_content(self):
        """Extract content from policy pages"""
        policy_details = {
            "text": "",
            "buttons": [],
            "links": []
        }
        
        try:
            # Try to find main content container
            for selector in ["main", "article", ".content", "#content", ".privacy-policy", ".cookie-policy"]:
                try:
                    content_element = self.driver.find_element(By.CSS_SELECTOR, selector)
                    if content_element:
                        policy_text = content_element.text
                        if len(policy_text) > 200:
                            policy_details["text"] = policy_text
                            
                            # Get links within policy
                            links = content_element.find_elements(By.TAG_NAME, "a")
                            for link in links[:10]:  # Limit to 10 links
                                if link.is_displayed():
                                    link_text = link.text.strip()
                                    link_href = link.get_attribute("href")
                                    if link_text and link_href:
                                        policy_details["links"].append({
                                            "text": link_text,
                                            "href": link_href,
                                            "element": link
                                        })
                            return policy_details
                except:
                    continue
            
            # Fallback to body
            body = self.driver.find_element(By.TAG_NAME, "body")
            policy_details["text"] = body.text[:5000] + "..." if len(body.text) > 5000 else body.text
            
        except Exception as e:
            print(f"❌ Error extracting policy content: {e}")
            policy_details["text"] = "Error extracting policy content"
        
        return policy_details
    
    def save_results(self, filename=None):
        """Save results to Excel"""
        if not self.data:
            print("❌ No data to save")
            return
        
        filename = filename or DATA_FILE
        
        # Clean data for Excel export
        cleaned_data = []
        for row in self.data:
            cleaned_row = {}
            for key, value in row.items():
                if key != 'element':  # Skip WebElement objects
                    cleaned_row[key] = value
            cleaned_data.append(cleaned_row)
        
        # Create DataFrame and save
        df = pd.DataFrame(cleaned_data)
        df.to_excel(filename, index=False)
        print(f"💾 Results saved to {filename}")
        
        # Display summary
        print(f"\n📊 Summary:")
        print(f"   Total states captured: {len(df)}")
        print(f"   Websites processed: {df['Website'].nunique()}")
        
        return df
    
    def close(self):
        """Close the driver"""
        try:
            # Make sure we're out of any iframe
            if self.current_iframe:
                self.driver.switch_to.default_content()
            self.driver.quit()
            print("🔌 Driver closed successfully")
        except:
            pass

# Example usage
def main():
    """Main execution function"""
    print("🚀 Enhanced Privacy Policy Scraper with Iframe Context Fix")
    print("=" * 60)
    
    # Initialize scraper
    scraper = IframeFixedPrivacyScraper(headless=False)
    
    # Website to test
    website = 'https://export.ebay.com/in/'
    #"https://www.dropbox.com"
    #'https://www.ebay.co.uk/'
    #'https://www.amazon.com'
    #'https://www.airbnb.com'
    #'https://www.theguardian.com'
    #"https://www.dropbox.com"
    
    try:
        # Run interactive exploration
        data = scraper.explore_interactively(website, max_depth=MAX_DEPTH)
        
        # Save results
        df = scraper.save_results()
        
        # Display preview
        if not df.empty:
            print("\n📋 Data Preview:")
            print(df[['Website', 'StateID', 'ChoiceMade', 'Choice Provided', 'In Iframe']].head())
        
    except KeyboardInterrupt:
        print("\n⚠️ Scraping interrupted by user")
    except Exception as e:
        print(f"\n❌ Scraping failed: {e}")
    finally:
        scraper.close()

if __name__ == "__main__":
    main()

🚀 Enhanced Privacy Policy Scraper with Iframe Context Fix
Driver initialized with user agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Ap...
🌐 Visiting https://export.ebay.com/in/...
🤖 Checking for bot detection...
📸 Initial screenshot: screenshots/export_ebay_com_initial_20250711_151256.png
Waiting up to 10s for consent banner to appear...
Found consent banner via id='onetrust-banner-sdk'
Found banner via selector: #onetrust-banner-sdk
Found banner via selector: .ot-sdk-container
🎯 Found 2 banner elements

--- Notice Text ---
By clicking "Accept all", you agree to the use of cookies and other technologies to process your personal data. We will also allow third-party companies listedhere, who we partner with, to store cookies on your device and use similar technologies to collect and use your personal data (e.g., IP address) for advertising personalisation, measurement and analytics, on and off our sites. You may decline consent by clicking "Decline all" or make granular choices